<a href="https://colab.research.google.com/github/osjayaprakash/notes/blob/main/colabs/sp500_stock_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S&P 500 Investment Strategy Analysis

In [1]:
# !pip3 install kagglehub pandas matplotlib

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Download SP500 stock for last 30 years

In [3]:
import kagglehub

path = kagglehub.dataset_download("asimislam/30-yrs-stock-market-data")
print("Path to dataset files:", path)

Path to dataset files: /Users/jp/.cache/kagglehub/datasets/asimislam/30-yrs-stock-market-data/versions/3


In [4]:
!ls "{path}"

30_yr_stock_market_data.csv 30_yr_stock_market_info.csv


In [5]:
df1 = pd.read_csv(f"{path}/30_yr_stock_market_data.csv")

In [6]:
df1.columns

Index(['Date', 'Dow Jones (^DJI)', 'Nasdaq (^IXIC)', 'S&P500 (^GSPC)',
       'NYSE Composite (^NYA)', 'Russell 2000 (^RUT)',
       'CBOE Volitility (^VIX)', 'DAX Index (^GDAXI)', 'FTSE 100 (^FTSE)',
       'Hang Seng Index (^HSI)', 'Cocoa (CC=F)', 'Coffee (KC=F)',
       'Corn (ZC=F)', 'Cotton (CT=F)', 'Live Cattle (LE=F)',
       'Orange Juice (OJ=F)', 'Soybeans (ZS=F)', 'Sugar (SB=F)',
       'Wheat (ZW=F)', 'Ethanol (EH=F)', 'Heating Oil (HO=F)',
       'Natural Gas (NG=F)', 'Crude Oil-Brent (BZ=F)', 'Crude Oil-WTI (CL=F)',
       'Copper (HG=F)', 'Gold (GC=F)', 'Palladium (PA=F)', 'Platinum (PL=F)',
       'Silver (SI=F)', 'Treasury Yield 5 Years (^FVX)',
       'Treasury Bill 13 Week (^IRX)', 'Treasury Yield 10 Years (^TNX)',
       'Treasury Yield 30 Years (^TYX)'],
      dtype='object')

In [7]:
df1.head(5)

,Date,Dow Jones (^DJI),Nasdaq (^IXIC),S&P500 (^GSPC),NYSE Composite (^NYA),Russell 2000 (^RUT),CBOE Volitility (^VIX),DAX Index (^GDAXI),FTSE 100 (^FTSE),Hang Seng Index (^HSI),...,Crude Oil-WTI (CL=F),Copper (HG=F),Gold (GC=F),Palladium (PA=F),Platinum (PL=F),Silver (SI=F),Treasury Yield 5 Years (^FVX),Treasury Bill 13 Week (^IRX),Treasury Yield 10 Years (^TNX),Treasury Yield 30 Years (^TYX)
0,1993-11-29,3677.800049,751.539978,461.899994,2697.040039,250.139999,14.12,2052.090088,3135.800049,9012.799805,...,NaN,NaN,NaN,NaN,NaN,NaN,5.075,3.11,5.732,6.226
1,1993-11-30,3683.949951,754.390015,461.790009,2694.080078,250.410004,13.76,2052.919922,3166.899902,9125.200195,...,NaN,NaN,NaN,NaN,NaN,NaN,5.146,3.14,5.795,6.286
2,1993-12-01,3697.080078,763.809998,461.890015,2699.159912,252.610001,13.83,2089.770020,3233.199951,9254.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,5.132,3.11,5.783,6.268
3,1993-12-02,3702.110107,766.729980,463.109985,2705.179932,252.910004,13.51,2099.760010,3223.899902,9238.200195,...,NaN,NaN,NaN,NaN,NaN,NaN,5.146,3.11,5.778,6.263
4,1993-12-03,3704.070068,772.219971,464.890015,2714.699951,253.860001,12.69,2128.300049,3234.199951,9294.400391,...,NaN,NaN,NaN,NaN,NaN,NaN,5.175,3.11,5.783,6.258


## Fill the missing values

use `ffill` to fill the data from previous days if it is missing. 

In [8]:
df = df1[['Date', 'S&P500 (^GSPC)']]
if 'S&P500 (^GSPC)' in df.columns:
    df = df.rename(columns={'S&P500 (^GSPC)': 'SP500'})
df.head(5)

if 'Date' in df.columns:
    df.set_index('Date', inplace=True)
    d_range = pd.date_range(start=df.index.min(), end=df.index.max())
    df.reindex(d_range)
    df.ffill(inplace=True)
    df = df.reset_index()
    df = df.rename(columns={'index': 'Date'})

missing_dates_df = df[df.isnull().any(axis=1)]
print(f"{missing_dates_df.size}, {df.size}, {len(df)}")

0, 15508, 7754


In [9]:
missing_dates_df.size
missing_dates_df.head(5)

,Date,SP500


In [10]:
# df['day'] = pd.to_datetime(df['Date']).dt.day
# df['month'] = pd.to_datetime(df['Date']).dt.month
# df['year'] = pd.to_datetime(df['Date']).dt.year

df['weekday'] = pd.to_datetime(df['Date']).dt.weekday
df['max'] = df['SP500'].expanding().max()
df['r7_max'] = df['SP500'].rolling(window=7).max()
df['r30_max'] = df['SP500'].rolling(window=30).max()

print((df['r7_max']*0.9>df['SP500']).sum())
print((df['r30_max']==df['SP500']).sum())

39
1367


In [11]:
df.query('r7_max*0.9>SP500')

,Date,SP500,weekday,max,r7_max,r30_max
1228,1998-08-31,957.280029,0,1186.750000,1092.849976,1165.069946
1649,2000-04-14,1356.560059,4,1527.459961,1516.349976,1527.459961
2020,2001-09-21,965.799988,4,1527.459961,1092.540039,1191.290039
2233,2002-07-22,819.849976,0,1527.459961,921.390015,1037.140015
2234,2002-07-23,797.700012,1,1527.459961,917.929993,1037.140015
3835,2008-09-29,1106.420044,0,1565.150024,1255.079956,1300.680054
3840,2008-10-06,1056.890015,0,1565.150024,1213.270020,1300.680054
3841,2008-10-07,996.229980,1,1565.150024,1166.359985,1300.680054
3842,2008-10-08,984.940002,2,1565.150024,1166.359985,1300.680054
3843,2008-10-09,909.919983,3,1565.150024,1161.060059,1282.829956


In [12]:
from collections import namedtuple

class Asset:
    def __init__(self, cash, stock):
        self.cash = cash
        self.stock = stock
        self.total = cash + stock

    def __add__(self, other):
        return Asset(self.cash + other.cash, self.stock + other.stock)


CurrentAsset = namedtuple('CurrentAsset', ['cash', 'stock'])
Inputs = namedtuple('Inputs', ['values', 'weekday', 'amount'])
Outputs = namedtuple('Outputs', ['final', 'history'])

def keep_cash(inputs):
    values, weekday, amount = inputs
    pa = CurrentAsset(cash=0, stock=0)
    final = CurrentAsset(cash=0,stock=0)
    history = []
    for d, v in zip(weekday, values):
        final = CurrentAsset(cash=final.cash+amount, stock=0)
        history += final
    return Outputs(final=final, history=history)

def dollar_avg(inputs):
    values, weekday, amount = inputs
    pa = CurrentAsset(cash=0, stock=0)
    final = CurrentAsset(cash=0,stock=0)
    history = []
    for d, v in zip(weekday, values):
        final = CurrentAsset(cash=0, stock=final.stock + (amount/v))
        history += final
    return Outputs(final=final, history=history)


v = dollar_avg(Inputs(values=df.SP500.to_list(), weekday=df.weekday.to_list(), amount=100))



In [13]:
print(v.final)

CurrentAsset(cash=0, stock=616.584752905432)
